In [2]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
import torch
from torch.utils.data import Dataset, DataLoader

class CharacterLevelShakespeareDataset(Dataset):
    def __init__(self, file_path, context_length):
        with open(file_path, 'r', encoding='utf-8') as file:
            raw_text = file.read()
        self.chars = sorted(list(set(raw_text)))
        self.char_to_index = {ch: i for i, ch in enumerate(self.chars)}
        self.index_to_char = {i: ch for ch, i in self.char_to_index.items()}
        self.vocab_size = len(self.chars)
        self.encoded_text = [self.char_to_index[ch] for ch in raw_text]
        self.context_length = context_length

    def __len__(self):
        return len(self.encoded_text) - self.context_length

    def __getitem__(self, index):
        input_sequence = self.encoded_text[index:index + self.context_length]
        target_sequence = self.encoded_text[index + 1:index + 1 + self.context_length]
        return torch.tensor(input_sequence, dtype=torch.long), torch.tensor(target_sequence, dtype=torch.long)

def build_shakespeare_dataloader(path='input.txt', context_length=100, batch_size=64):
    dataset = CharacterLevelShakespeareDataset(path, context_length)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    return dataloader, dataset.vocab_size, dataset.char_to_index, dataset.index_to_char

dataloader, vocab_size, char_to_index, index_to_char = build_shakespeare_dataloader()


--2025-04-15 16:32:13--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.07s   

2025-04-15 16:32:13 (14.6 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [13]:
from torch import nn
class Rnn(nn.Module):
  def __init__(self, vocab_size, embedding_dim=1, hidden_dim=5, output_dim=1):
    super().__init__()
    self.hidden_trans = nn.Linear(hidden_dim, hidden_dim)
    self.input_trans = nn.Linear(embedding_dim, hidden_dim)
    self.output_trans = nn.Linear(hidden_dim, output_dim)
    self.hidden_activation = nn.Tanh()
    self.embedding_layer = nn.Embedding(vocab_size, embedding_dim)
  def forward(self, x):
    batch_size, sequence_length = x.shape
    hidden_state = torch.zeros((batch_size, self.hidden_trans.in_features))
    for i in range(0, sequence_length):
      tokens = x[:,i]
      embedding = self.embedding_layer(tokens)
      hidden_state = self.hidden_activation(self.input_trans(embedding) + self.hidden_trans(hidden_state))
    return self.output_trans(hidden_state)

In [19]:
import torch
from torch import nn

class Rnn(nn.Module):
    def __init__(self, vocab_size, embedding_dim=1, hidden_dim=5, output_dim=1):
        super().__init__()
        self.hidden_trans = nn.Linear(hidden_dim, hidden_dim)
        self.input_trans = nn.Linear(embedding_dim, hidden_dim)
        self.output_trans = nn.Linear(hidden_dim, output_dim)
        self.hidden_activation = nn.Tanh()
        self.embedding_layer = nn.Embedding(vocab_size, embedding_dim)

    def forward(self, x):
        batch_size, sequence_length = x.shape
        hidden_state = torch.zeros((batch_size, self.hidden_trans.in_features), device=x.device)
        outputs = []
        for i in range(sequence_length):
            tokens = x[:, i]
            embedding = self.embedding_layer(tokens)
            hidden_state = self.hidden_activation(
                self.input_trans(embedding) + self.hidden_trans(hidden_state)
            )
            logits = self.output_trans(hidden_state)
            outputs.append(logits.unsqueeze(1))
        return torch.cat(outputs, dim=1)

In [17]:
from torch import nn
EPOCHS=1
loss = nn.CrossEntropyLoss()
model = Rnn(vocab_size, output_dim=vocab_size)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
for epoch in range(0, EPOCHS):
  for input_sequence, output_sequence in dataloader:
    output = model(input_sequence)
    print(output.shape, output_sequence.shape)
    output = loss(output, output_sequence)
    optimizer.zero_grad()
    output.backward()
    optimizer.step()

torch.Size([64, 65]) torch.Size([64, 100])


RuntimeError: 0D or 1D target tensor expected, multi-target not supported

In [20]:
EPOCHS = 1
model = Rnn(vocab_size, output_dim=vocab_size)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(EPOCHS):
    for input_sequence, output_sequence in dataloader:
        output_logits = model(input_sequence)                          # [batch_size, seq_len, vocab_size]
        predicted = output_logits.reshape(-1, output_logits.shape[-1]) # [batch_size*seq_len, vocab_size]
        target = output_sequence.reshape(-1)                           # [batch_size*seq_len]

        loss = loss_function(predicted, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

IndexError: index out of range in self

In [18]:
from torch import nn
import torch

class Rnn(nn.Module):
  def __init__(self, vocab_size, embedding_dim=1, hidden_dim=5, output_dim=1):
    super().__init__()
    self.hidden_trans = nn.Linear(hidden_dim, hidden_dim)
    self.input_trans = nn.Linear(embedding_dim, hidden_dim)
    self.output_trans = nn.Linear(hidden_dim, output_dim)
    self.hidden_activation = nn.Tanh()
    self.embedding_layer = nn.Embedding(vocab_size, embedding_dim)

  def forward(self, x):
    batch_size, sequence_length = x.shape
    hidden_state = torch.zeros((batch_size, self.hidden_trans.in_features), device=x.device)
    for i in range(sequence_length):
      tokens = x[:, i]
      embedding = self.embedding_layer(tokens)
      hidden_state = self.hidden_activation(self.input_trans(embedding) + self.hidden_trans(hidden_state))
    return self.output_trans(hidden_state)

# Training
EPOCHS = 1
vocab_size = 30  # define this properly
model = Rnn(vocab_size, embedding_dim=8, hidden_dim=32, output_dim=vocab_size)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(EPOCHS):
  for input_sequence, output_sequence in dataloader:
    input_sequence = input_sequence.to(model.output_trans.weight.device)
    output_sequence = output_sequence.to(model.output_trans.weight.device)

    output = model(input_sequence)  # output.shape = (batch_size, vocab_size)
    loss = loss_function(output, output_sequence)  # output_sequence.shape = (batch_size,)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


IndexError: index out of range in self

In [35]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

class CharacterLevelShakespeareDataset(Dataset):
    def __init__(self, file_path, context_length):
        with open(file_path, 'r', encoding='utf-8') as file:
            raw_text = file.read()
        self.chars = sorted(list(set(raw_text)))
        self.char_to_index = {ch: i for i, ch in enumerate(self.chars)}
        self.index_to_char = {i: ch for ch, i in self.char_to_index.items()}
        self.vocab_size = len(self.chars)
        self.encoded_text = [self.char_to_index[ch] for ch in raw_text]
        self.context_length = context_length

    def __len__(self):
        return len(self.encoded_text) - self.context_length

    def __getitem__(self, index):
        input_sequence = self.encoded_text[index:index + self.context_length]
        target_sequence = self.encoded_text[index + 1:index + 1 + self.context_length]
        return torch.tensor(input_sequence, dtype=torch.long), torch.tensor(target_sequence, dtype=torch.long)

def build_shakespeare_dataloader(path='input.txt', context_length=100, batch_size=64):
    dataset = CharacterLevelShakespeareDataset(path, context_length)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    return dataloader, dataset.vocab_size, dataset.char_to_index, dataset.index_to_char

dataloader, vocab_size, char_to_index, index_to_char = build_shakespeare_dataloader()


import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
class Rnn(nn.Module):
    def __init__(self, vocab_size, embedding_dim=64, hidden_dim=128, output_dim=None):
        super().__init__()
        self.hidden_trans = nn.Linear(hidden_dim, hidden_dim)
        self.input_trans = nn.Linear(embedding_dim, hidden_dim)
        self.output_trans = nn.Linear(hidden_dim, vocab_size)
        self.hidden_activation = nn.Tanh()
        self.embedding_layer = nn.Embedding(vocab_size, embedding_dim)
    def forward(self, x):
        batch_size, sequence_length = x.shape
        hidden_state = torch.zeros((batch_size, self.hidden_trans.in_features), device=x.device)
        outputs = []
        for i in range(sequence_length):
            tokens = x[:, i]
            embedding = self.embedding_layer(tokens)
            hidden_state = self.hidden_activation(
                self.input_trans(embedding) + self.hidden_trans(hidden_state)
            )
            logits = self.output_trans(hidden_state)
            outputs.append(logits.unsqueeze(1))
        return torch.cat(outputs, dim=1)

EPOCHS = 1
embedding_dim = 64
hidden_dim = 128
model = Rnn(vocab_size, embedding_dim=embedding_dim, hidden_dim=hidden_dim, output_dim=vocab_size)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
from tqdm import tqdm
for epoch in range(EPOCHS):
    for input_sequence, output_sequence in tqdm(dataloader):
        output_logits = model(input_sequence)
        predicted = output_logits.reshape(-1, output_logits.shape[-1])
        target = output_sequence.reshape(-1)
        loss = loss_function(predicted, target)
        optimizer.zero_grad()
        loss.backward()
        print(loss)
        optimizer.step()

--2025-04-15 17:57:00--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2025-04-15 17:57:00 (18.7 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



  0%|          | 2/17426 [00:00<48:45,  5.96it/s]  

tensor(4.2253, grad_fn=<NllLossBackward0>)
tensor(4.1804, grad_fn=<NllLossBackward0>)


  0%|          | 5/17426 [00:00<31:36,  9.19it/s]

tensor(4.1234, grad_fn=<NllLossBackward0>)
tensor(4.0607, grad_fn=<NllLossBackward0>)
tensor(4.0055, grad_fn=<NllLossBackward0>)


...

100%|█████████▉| 17425/17426 [32:03<00:00,  7.20it/s]

tensor(1.5183, grad_fn=<NllLossBackward0>)
tensor(1.5273, grad_fn=<NllLossBackward0>)


100%|██████████| 17426/17426 [32:04<00:00,  7.06it/s]

tensor(1.4895, grad_fn=<NllLossBackward0>)


100%|██████████| 17426/17426 [32:04<00:00,  9.06it/s]


In [56]:
torch.argmax(model(input_sequence)[0][8])

tensor(41)

In [55]:
target[8:]

tensor([41, 46,  1,  ..., 56, 43,  1])

In [57]:
torch.argmax(model(input_sequence)[0][9])

tensor(46)

In [58]:
torch.argmax(model(input_sequence)[0][10])

tensor(1)